In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cargar datos

In [ ]:
from typing import List, Union
from pathlib import Path
from torch.utils.data import random_split

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim
import time
import csv
import os

dataset = datasets.ImageFolder('/kaggle/input/iais22-birds/birds/birds/', transform=transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()]))
train_set, val_set = random_split(dataset, (int(len(dataset) * 0.9) + 1, int(len(dataset) * 0.1)))

In [ ]:
train = DataLoader(dataset, batch_size=200, shuffle=True)
train_dataloader = DataLoader(train_set, batch_size=200, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=200, shuffle=True)

# Creación del modelo

In [ ]:

channel1 = 16
channel2 = 32

class NET(nn.Module):
    def __init__(self):
        super(NET, self).__init__()
        self.cnn = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=channel1,kernel_size=3, padding=1),
                         nn.ReLU(),
                         nn.Conv2d(in_channels=channel1, out_channels=channel2,kernel_size=3, padding=1),
                         nn.ReLU(),
                         nn.MaxPool2d(2, 2),
                         nn.Flatten(),
                         nn.Linear(in_features=32*32*channel2, out_features=400))
        
    def forward(self, x):
        x = self.cnn(x)
        return x
    


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
model = NET().to(device)
print(model)

# Entrenamiento de la red

In [ ]:
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=learning_rate, amsgrad=False)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    


epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train, model, loss_fn, optimizer)
    test_loop(val_dataloader, model, loss_fn)
print("Done!")

# GUARDAR MODELO

In [ ]:
PATH = './red_convolucional.pth'
torch.save(model.state_dict(), PATH)

# CARGAR MODELO

In [ ]:
red = NET()
red.load_state_dict(torch.load(PATH))

# CREAR CSV

In [ ]:
from PIL import Image
class BirdsDatasetTest(torch.utils.data.Dataset):
    def __init__(self, path: Union[Path, str],
                transform: Union['Transform', List['Transform']] = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])):
        self.path = Path(path)
        self.labels = [p.name for p in path.glob('x')]
        self.images = list(path.glob('*/*.jpg'))
        self.transform = transform
        
    
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self, index:int) :
        image_path = self.images[index]
        image = self.transform(Image.open(str(image_path)))
        archivo = os.path.basename(image_path)
        id = archivo.split(sep=".")[0]
        return image,id

In [ ]:
test_dat = BirdsDatasetTest(path = Path('/kaggle/input/iais22-birds/submission_test/'))
clases = dataset.classes
dataloader = DataLoader(test_dat, batch_size=1)

In [ ]:
trans = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
with open('submission.csv', 'w') as csvfile:
    fieldnames = ['Id', 'Category']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for image,id in dataloader:
        pred = red(image)
        categoria = clases[pred.argmax(1)]
        writer.writerow({'Id': int(id[0]), 'Category': categoria})